In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import pylab as plt
import json
import time
import requests
import warnings
from bs4 import BeautifulSoup
from tqdm import tqdm
import requests
from joblib import Parallel, delayed
from tqdm_joblib import tqdm_joblib
import plotly.express as px
from tqdm.autonotebook import tqdm
warnings.filterwarnings('ignore')


                Extracción, transformación y limpieza de datos del dataset de películas, series y documentales de Netflix

In [3]:
film = pd.read_csv('../data/Net_titles.csv', encoding='utf-8', encoding_errors='ignore')
actor = pd.read_csv('../data/Net_credits.csv', encoding='utf-8', encoding_errors='ignore') 

In [4]:
# Para rellenar los valores nulos de la columna Age_certification. Si pertenece a los siguientes géneros se asumirá que la certificación de edad es R (+17)
def fill_age_null(row):
    if pd.isna(row['age_certification']):
        if 'thriller' in row['genres'] or 'horror' in row['genres'] or 'black comedy' in row['genres']:
            return 'R'
    return row['age_certification']

film['age_certification'] = film.apply(lambda row: fill_age_null(row), axis=1)

In [5]:
# Eliminamos los valores nulos de las columnas relacionados con IMDB y TMDB, ya que sin el imdb_id es imposible analizarlos
# Rellenamos los valores nulos de la columna seasons con '-1'
# Rellenamos los valores nulos de la columna character del df 'actor' con la palabra 'unknown'
# Rellenamos los valores nulos de la columna description con 'unknown'
# Rellenamos los valores nulos de la columna age_certification con 'unknown'

film.dropna(subset=['imdb_id', 'imdb_score', 'imdb_votes', 'tmdb_popularity', 'tmdb_score'], inplace=True)
film['age_certification'] = film['age_certification'].fillna('unknown')
film['seasons'] = film['seasons'].fillna('-1')
film['description'] = film['description'].fillna('unknown')
actor['character'] = actor['character'].fillna('unknown')



In [8]:
from joblib import Parallel, delayed
from tqdm import tqdm

def extract_reviews_and_ratings(imdb_id):
    url = f'https://www.imdb.com/title/{imdb_id}/reviews?ref_=tt_ov_rt' 
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        reviews = soup.find_all('div', {'class': 'text show-more__control'})  
        ratings = soup.find_all('span', {'class': 'rating-other-user-rating'})
        results = []                                        # crea la lista para almacenar los resultados
        for rev, rat in zip(reviews, ratings):              # itera sobre reviews y rating a la vez
            rev_div = rev.find_parent('div', {'class': 'lister-item-content'})  
            rev_title = rev_div.find('a', {'class': 'title'}).text.strip() if rev_div.find('a', {'class': 'title'}) else 'No title'
            rat_title = rat.text.strip() if rat else 'Sin calificación'
            results.append((imdb_id, rev_title, rat_title))
        return results
    else:
        print(f'Error al obtener comentarios de la película con IMDB ID {imdb_id}')
        return []

if __name__ == '__main__':
    imdb_ids = film['imdb_id'].tolist()                                                                                         #pasamos la columna del df a lista
    reviews_and_ratings = Parallel(n_jobs=-1)(delayed(extract_reviews_and_ratings)(imdb_id) for imdb_id in tqdm(imdb_ids))      #extraemos en paralelo las reviews y ratings de cada id
    reviews_and_ratings = [review for sublist in reviews_and_ratings for review in sublist]                                     #aplanamos la lista de listas 'reviews_and_ratings'
    com_rev = pd.DataFrame(reviews_and_ratings, columns=['id', 'review_title', 'rating_title'])



100%|██████████| 5131/5131 [14:34<00:00,  5.87it/s]


In [20]:
#com_rev.to_csv('../data/Net_comments.csv', index=False)
#film.to_csv('../data/Net_titles_clean.csv', index=False)
#actor.to_csv('../data/Net_actors_clean.csv', index=False)

                Realizamos el mismo proceso con las tablas de HBO

In [24]:
film = pd.read_csv('../data/HBO_titles.csv', encoding='utf-8', encoding_errors='ignore')
actor = pd.read_csv('../data/HBO_credits.csv', encoding='utf-8', encoding_errors='ignore') 

In [26]:
def fill_age_null(row):
    if pd.isna(row['age_certification']):
        if 'thriller' in row['genres'] or 'horror' in row['genres'] or 'black comedy' in row['genres']:
            return 'R'
    return row['age_certification']

film['age_certification'] = film.apply(lambda row: fill_age_null(row), axis=1)

In [28]:
film.dropna(subset=['imdb_id', 'imdb_score', 'imdb_votes', 'tmdb_popularity', 'tmdb_score'], inplace=True)
film['age_certification'] = film['age_certification'].fillna('unknown')
film['seasons'] = film['seasons'].fillna('-1')
film['description'] = film['description'].fillna('unknown')
actor['character'] = actor['character'].fillna('unknown')

In [31]:
from joblib import Parallel, delayed
from tqdm import tqdm

def extract_reviews_and_ratings(imdb_id):
    url = f'https://www.imdb.com/title/{imdb_id}/reviews?ref_=tt_ov_rt' 
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        reviews = soup.find_all('div', {'class': 'text show-more__control'})  
        ratings = soup.find_all('span', {'class': 'rating-other-user-rating'})
        results = []
        for rev, rat in zip(reviews, ratings):
            rev_div = rev.find_parent('div', {'class': 'lister-item-content'})  
            rev_title = rev_div.find('a', {'class': 'title'}).text.strip() if rev_div.find('a', {'class': 'title'}) else 'No title'
            rat_title = rat.text.strip() if rat else 'Sin calificación'
            results.append((imdb_id, rev_title, rat_title))
        return results
    else:
        print(f'Error al obtener comentarios de la película con IMDB ID {imdb_id}')
        return []

if __name__ == '__main__':
    imdb_ids = film['imdb_id'].tolist()
    reviews_and_ratings = Parallel(n_jobs=-1)(delayed(extract_reviews_and_ratings)(imdb_id) for imdb_id in tqdm(imdb_ids))
    reviews_and_ratings = [review for sublist in reviews_and_ratings for review in sublist]
    com_rev = pd.DataFrame(reviews_and_ratings, columns=['id', 'review_title', 'rating_title'])

100%|██████████| 2606/2606 [12:14<00:00,  3.55it/s]


In [32]:
#com_rev.to_csv('../data/HBO_comments.csv', index=False)
#film.to_csv('../data/HBO_titles_clean.csv', index=False)
#actor.to_csv('../data/HBO_actors_clean.csv', index=False)

In [10]:
# Creamos el df de actores
actor = pd.read_csv('../data/HBO_actors_clean.csv', encoding='utf-8', encoding_errors='ignore') 

In [11]:
actor = actor.drop(['id','character', 'role'], axis= 1)

In [12]:
actor = actor.drop_duplicates()

In [13]:
actor.to_csv('../data/sql/HBO_actors_sql.csv', index=False)

In [8]:
# Creamos el df de reparto
cast = pd.read_csv('../data/HBO_actors_clean.csv', encoding='utf-8', encoding_errors='ignore')

In [9]:
cast = cast.drop(['name'], axis= 1)

In [10]:
cast.rename(columns={'id': 'film_id'}, inplace=True)


In [11]:
cast['id'] = range(1, len(cast)+1)

In [12]:
cast = cast.reindex(columns=['id', 'person_id', 'film_id', 'character', 'role'])


In [14]:
cast.to_csv('../data/sql/HBO_cast_sql.csv', index=False)

In [15]:
# Creamos el df de reviews
com = pd.read_csv('../data/HBO_comments.csv', encoding='utf-8', encoding_errors='ignore')

In [16]:
com.rename(columns={'id': 'imdb_id'}, inplace=True)

In [17]:
com['id'] = range(1, len(com)+1)

In [18]:
com = com.reindex(columns=['id', 'imdb_id', 'review_title', 'rating_title'])

In [19]:
com.to_csv('../data/sql/HBO_reviews_sql.csv', index=False)

In [20]:
# Creamos el df de films
film = pd.read_csv('../data/HBO_titles_clean.csv', encoding='utf-8', encoding_errors='ignore') 
film.rename(columns={'id': 'film_id'}, inplace=True)
film.to_csv('../data/sql/HBO_titles_sql.csv', index=False)


In [21]:
rev = pd.read_csv('../data/sql/HBO_reviews_sql.csv', encoding='utf-8', encoding_errors='ignore') 
rev.rename(columns={'id': 'review_id'}, inplace=True)
rev.to_csv('../data/sql/HBO_reviews_sql.csv', index=False)

                Visualización de datos

In [22]:
film = pd.read_csv('../data/Net_titles_clean.csv', encoding='utf-8', encoding_errors='ignore')

In [5]:
# Definimos la función filtrar_gen para quitar del gráfico aquellos géneros y subgéneros que representen una minoría
def filtrar_gen(df, col, thresh):
    counts = df[col].value_counts()
    mask = df[col].isin(counts[counts > thresh].index)
    return df[mask]

In [24]:
# Vamos a crear un nuevo df cuyas columnas sean el género y el subgénero de de las películas
film['genres'] = film['genres'].str.strip('[]').str.replace("'", '').str.split(', ')
film_gen = film['genres'].apply(pd.Series).rename(columns = lambda x : 'genre_' + str(x))
df_sunburst = pd.concat([film_gen['genre_0'], film_gen['genre_1'], film_gen['genre_2']], axis=1)
df_sunburst.columns = ['Level 1', 'Level 2', 'Level 3']


In [25]:
# Eliminamos los nulos de los subgéneros, ya que representan que género y subgénero coinciden

df_sunburst = df_sunburst.dropna(subset=['Level 3'])
df_sunburst = df_sunburst.dropna(subset=['Level 2'])



In [26]:
# definimos un thresh con un umbral de 50 valores únicos para no tener en cuenta aquellos géneros y subgéneros que son poco representativos y que molestan visualmente
df_sunburst = filtrar_gen(df_sunburst, 'Level 2', 50)   
df_sunburst = filtrar_gen(df_sunburst, 'Level 1', 50)

In [27]:
fig = px.sunburst(df_sunburst, path=['Level 1', 'Level 2'], width=600, height=600)
fig.show()
fig.write_html('../images/NET_gen_sunburst.html')


In [28]:
film = pd.read_csv('../data/HBO_titles_clean.csv', encoding='utf-8', encoding_errors='ignore')

film['genres'] = film['genres'].str.strip('[]').str.replace("'", '').str.split(', ')
film_gen = film['genres'].apply(pd.Series).rename(columns = lambda x : 'genre_' + str(x))

df_sunburst = pd.concat([film_gen['genre_0'], film_gen['genre_1'], film_gen['genre_2']], axis=1)
df_sunburst.columns = ['Level 1', 'Level 2', 'Level 3']

df_sunburst = df_sunburst.dropna(subset=['Level 3'])
df_sunburst = df_sunburst.dropna(subset=['Level 2'])

df_sunburst = filtrar_gen(df_sunburst, 'Level 2', 40)  
df_sunburst = filtrar_gen(df_sunburst, 'Level 1', 40)

fig = px.sunburst(df_sunburst, path=['Level 1', 'Level 2'], width=600, height=600)

fig.show()
fig.write_html('../images/HBO_gen_sunburst.html')

In [25]:
# Vamos a unir ambos dataframes y crear una columna que indicque la plataforma de la plicula
hbo = pd.read_csv('../data/sql/HBO_titles_sql.csv', encoding='utf-8', encoding_errors='ignore') 
net = pd.read_csv('../data/sql/Net_titles_sql.csv', encoding='utf-8', encoding_errors='ignore')  

both_platforms = set(net['film_id']).intersection(set(hbo['film_id']))

# Agrega una columna "both_platforms" al dataframe combinado
net['both_platforms'] = net['film_id'].isin(both_platforms)
hbo['both_platforms'] = hbo['film_id'].isin(both_platforms)

net['platform'] = 'Netflix'
hbo['platform'] = 'HBO'

df_combined = pd.concat([net, hbo])


In [27]:
df_combined.loc[df_combined['both_platforms'], 'platform'] = 'both platforms'

In [31]:
df_combined = df_combined.drop('both_platforms', axis=1)

In [33]:
df_combined.to_csv('../data/sql/group/films.csv', index=False)

In [3]:
df_combined = pd.read_csv('../data/sql/group/films.csv', encoding='utf-8', encoding_errors='ignore') 

In [7]:
# grafico sunburst de las dos plataformas en conjunto
df_combined['genres'] = df_combined['genres'].astype(str).str.strip('[]').str.replace("'", '').str.split(', ')
film_gen = df_combined['genres'].apply(pd.Series).rename(columns = lambda x : 'genre_' + str(x))

df_sunburst = pd.concat([film_gen['genre_0'], film_gen['genre_1'].fillna('N/A')], axis=1)
df_sunburst.columns = ['Level 1', 'Level 2']

df_sunburst = filtrar_gen(df_sunburst, 'Level 2', 40)  
df_sunburst = filtrar_gen(df_sunburst, 'Level 1', 40)

fig = px.sunburst(df_sunburst, path=['Level 1', 'Level 2'], width=600, height=600)

fig.show()

In [ ]:
film_gen.drop(['genre_2', 'genre_3','genre_4', 'genre_5', 'genre_6', 'genre_7', 'genre_8', 'genre_9'], axis=1, inplace=True)

In [13]:
film_gen['genre_1'] = film_gen['genre_1'].fillna('N/A')

In [15]:
film_gen.rename(columns={'genre_0':'genre', 'genre_1':'subgenre'})

,genre,subgenre
0,drama,crime
1,drama,action
2,fantasy,action
3,war,action
4,comedy,european
...,...,...
7732,comedy,N/A
7733,comedy,animation
7734,music,N/A
7735,comedy,N/A


In [17]:
df_combined = df_combined.drop('genres', axis=1)

In [19]:
df_combined = pd.concat([df_combined, film_gen], axis=1)

In [23]:
df_combined = df_combined.reindex(columns=['film_id', 'title', 'type', 'description', 'genre_0', 'genre_1', 'release_year', 'age_certification', 'runtime', 'production_countries', 'seasons', 'imdb_id', 'imdb_score', 'imdb_votes', 'tmdb_popularity', 'tmdb_score', 'platform'])

In [27]:
df_combined = df_combined.rename(columns={'genre_0':'genre', 'genre_1':'subgenre'})

In [28]:
df_combined

,film_id,title,type,description,genre,subgenre,release_year,age_certification,runtime,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,platform
0,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,drama,crime,1976,R,114,['US'],-1.0,tt0075314,8.2,808582.0,40.965,8.179,both platforms
1,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,drama,action,1972,R,109,['US'],-1.0,tt0068473,7.7,107673.0,10.010,7.300,Netflix
2,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",fantasy,action,1975,PG,91,['GB'],-1.0,tt0071853,8.2,534486.0,15.461,7.811,Netflix
3,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,war,action,1967,unknown,150,"['GB', 'US']",-1.0,tt0061578,7.7,72662.0,20.398,7.600,both platforms
4,ts22164,Monty Python's Flying Circus,SHOW,A British sketch comedy series with the shows ...,comedy,european,1969,TV-14,30,['GB'],4.0,tt0063929,8.8,73424.0,17.617,8.306,Netflix
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7732,tm1310730,Marlon Wayans: God Loves Me,MOVIE,"Filmed at the Center Stage Theater in Atlanta,...",comedy,N/A,2023,unknown,60,['US'],-1.0,tt26753138,6.3,204.0,15.338,6.700,HBO
7733,ts171230,Poor Devil,SHOW,A typical 18-year-old who happens to be the An...,comedy,animation,2023,TV-MA,22,['ES'],1.0,tt15764846,6.6,265.0,13.511,7.714,HBO
7734,tm1306271,The Weeknd: Live at SoFi Stadium,MOVIE,"Filmed at LA’s SoFi Stadium, The Weeknd brings...",music,N/A,2023,R,98,['US'],-1.0,tt26685153,8.1,257.0,23.370,5.800,HBO
7735,tm1305288,"Marcella Arguello: Bitch, Grow Up!",MOVIE,Arguello couples her larger-than-life stage pr...,comedy,N/A,2023,R,37,['US'],-1.0,tt26623699,6.9,27.0,7.509,2.000,HBO


In [30]:
df_combined.to_csv('../data/sql/group/films.csv', index=False)

In [31]:
df_combined = pd.read_csv('../data/sql/group/films.csv', encoding='utf-8', encoding_errors='ignore') 

In [34]:
df_combined['subgenre'] = df_combined['subgenre'].fillna('-')

In [36]:
df_combined.to_csv('../data/sql/group/films.csv', index=False)